In [6]:
import json, sys, os, requests
import altair as alt
from altair import expr, datum
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [115]:
def get_ons_data_ts(dataset, series, save=True, api_path="https://api.ons.gov.uk"):
    r = requests.get(
        api_path
        + "/timeseries/"
        + series.lower()
        + "/dataset/"
        + dataset.lower()
        + "/data"
    )
    content = json.loads(r.content)
    if save:
        print("Saving DATASET", dataset, "SERIES", series, "to local mirror...")
        open("datasets/ons/" + dataset + "/" + series + ".json", "w").write(
            json.dumps(content)
        )
        print("OK!")
    return content

In [128]:
dataset = "lms"
data={}
for series in ["mgsx", "lf24","kac3"]:
    d = get_ons_data_ts(dataset, series)
    data[d['description']['title']]=d['months']

Saving DATASET lms SERIES mgsx to local mirror...
OK!
Saving DATASET lms SERIES lf24 to local mirror...
OK!
Saving DATASET lms SERIES kac3 to local mirror...
OK!


In [130]:
data.keys()

dict_keys(['Unemployment rate (aged 16 and over, seasonally adjusted)', 'Employment rate (aged 16 to 64, seasonally adjusted)', 'AWE: Whole Economy Year on Year Three Month Average Growth (%): Seasonally Adjusted Total Pay Excluding Arrears'])

In [120]:
live = False
if live:
    url = (
        "https://api.allorigins.win/raw?url=https://api.ons.gov.uk/timeseries/"
        + series
        + "/dataset/"
        + dataset
        + "/data"
    )
else:
    url = (
        "https://raw.githubusercontent.com/EconomicsObservatory/ECOdataHUB/main/datasets/ons/"
        + dataset
        + "/"
        + series
        + ".json"
    )
base = alt.Chart(
    alt.Data(
        url=url,
        format=alt.DataFormat(type="json", property="months"),
    )
).encode(x="date:T")
line = base.mark_line(interpolate="basis").encode(
    y="value:Q",
    #     color='category:N'
)

# Create a selection that chooses the nearest point & selects based on x-value
nearest = alt.selection(
    type="single", nearest=True, on="mouseover", fields=["date"], empty="none"
)
# Transparent selectors across the chart. This is what tells us
# the x-value of the cursor
selectors = (
    base.mark_point()
    .encode(
        opacity=alt.value(0),
    )
    .add_selection(nearest)
)

# Draw points on the line, and highlight based on selection
points = line.mark_point().encode(
    opacity=alt.condition(nearest, alt.value(1), alt.value(0))
)

# Draw text labels near the points, and highlight based on selection
text = line.mark_text(align="left", dx=-25, dy=35).encode(
    text=alt.condition(nearest, "value:Q", alt.value(" "))
)

# Draw a rule at the location of the selection
rules = base.mark_rule(color="gray").transform_filter(nearest)

# Put the five layers into a chart and bind the data
layer1 = alt.layer(line, selectors, points, rules, text).properties(
    width=600, height=300
)
layer1.save("panels/" + dataset + ".json")
layer1

alt.LayerChart(...)